In [ ]:
from src.anaconda.ae5.secrets import load_ae5_user_secrets
from dotenv import load_dotenv

# load defined environmental variables
load_dotenv(dotenv_path="../env/.env.ae5")
load_ae5_user_secrets()

In [ ]:
from src.anaconda.mlflow.service.factory import build_mlflow_client, build_client
from src.anaconda.mlflow.client import AnacondaMlFlowClient

client: AnacondaMlFlowClient = build_client(client=build_mlflow_client())

In [ ]:
client.prune(pruneables=client.get_pruneables(), dry_run=False)